In [25]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from tqdm import tqdm
warnings.filterwarnings("ignore")
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [73]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

In [27]:
file_train_name='./Benz_data/train.csv'
file_test_name='./Benz_data/test.csv'

In [62]:
Train=pd.read_csv(file_train_name,index_col='ID')
Test=pd.read_csv(file_test_name,index_col='ID')
target=Train['y']
Train.drop(['y'],axis=1,inplace=True)

In [63]:
print("Train shape",Train.shape)
print("Test shape",Test.shape)

Train shape (4209, 376)
Test shape (4209, 376)


In [64]:
print("feature number %d"%(Train.shape[1]))
print('-----------------------------------')
Train.dtypes.value_counts()

feature number 376
-----------------------------------


int64     368
object      8
dtype: int64

In [65]:
feature=[[],[],[],[]]
for c in Train.columns:
    t=Train[c].dtype
    u=len(np.unique(Train[c]))
    if t==object:
        feature[0].append(c)
    elif u==1:
        feature[1].append(c)
    elif u==2:
        feature[2].append(c)
    else:
        feature[3].append(c)
    
print("String:%d\nConstant:%d\n2 kind:%d\nother kind:%d"%(len(feature[0]),len(feature[1]),len(feature[2]),len(feature[3])))
print("String features:",feature[0])
print("Constant features:",feature[1])

String:8
Constant:12
2 kind:356
other kind:0
String features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']
Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [68]:
Full=Train.append(Test)

In [104]:
new_Full=Full.drop(feature[1],axis=1)

pre=OneHotEncoder(sparse=False)
pre.fit(new_Full[feature[0]])
tmp=pre.transform(new_Full[feature[0]])
pre_Full=pd.concat([new_Full,pd.DataFrame(tmp,index=new_Full.index)],axis=1)
Last_Full=pre_Full.drop(feature[0],axis=1)

Train,Test=Last_Full[:4209],Last_Full[4209:]

In [145]:
params={
        'learning_rate': 0.1, 
        'n_estimators': 55, 
        'max_depth': 3, 
        'min_child_weight': 7, 
        'seed': 0,
        'subsample': 0.7, 
        'colsample_bytree': 0.9, 
        'gamma': 0.001, 
        'reg_alpha': 0.2, 
        'reg_lambda': 2
}

# cv_params={
#     'reg_lambda': [1,2,3]
# }

reg=xgb.XGBRegressor(**params)
# optimized_GBM=GridSearchCV(estimator=reg, param_grid=cv_params, scoring='r2', cv=30,verbose=1, n_jobs=-1)
# optimized_GBM.fit(Train,target)
# print(optimized_GBM.best_score_)
# print(optimized_GBM.best_params_)    
reg.fit(Train,target)

[21:05:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.9, gamma=0.001,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=7, missing=None, n_estimators=55,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0.2, reg_lambda=2, scale_pos_weight=1, seed=0,
       silent=None, subsample=0.7, verbosity=1)

In [146]:
csv=pd.DataFrame({'ID':Test.index,'y':reg.predict(Test)})
csv.to_csv('XXX.csv',index=False)